### ผลการทดลองที่ 2

1. เขียนโปรแกรมเพื่อเพิ่มข้อมูลนักศึกษา 5 คน รันโปรแกรมและบันทึกรูปผลการรันโปรแกรม

In [7]:
class BTreeNode:
    def __init__(self, leaf=True):
        self.leaf = leaf  # เป็นใบหรือไม่
        self.keys = []  # เก็บค่าของโหนด (เรียงจากน้อยไปมาก)
        self.data = []  # ข้อมูลที่สัมพันธ์กับ key
        self.children = []  # ลูกของโหนด (ถ้าไม่ใช่ใบ)

class BTree:
    def __init__(self, order):
        self.root = None  # โหนดราก
        self.order = order  # ระดับของ B-Tree

    def get_max_keys(self):
        return self.order - 1  # จำนวน key สูงสุดในโหนด

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode()
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.get_max_keys():
            new_root = BTreeNode(leaf=False)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.get_max_keys():
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf)
        mid = (order - 1) // 2  # คำนวณค่ากลางที่ถูกต้อง

        # ดึงค่า mid ขึ้นไป parent
        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        # ก๊อบปี้ค่าที่เหลือไปยังโหนดใหม่
        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        # ตัดค่าที่เคลื่อนย้ายออกจากโหนดเดิม
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def traverse(self, node=None, level=0):
        if node is None:
            node = self.root
        print("Level", level, "Keys:", node.keys)
        if not node.leaf:
            for child in node.children:
                self.traverse(child, level + 1)

# เพิ่มข้อมูลนักศึกษา
btree = BTree(3)
students = [(192, "สุภัทรา"), (130, "พีรพัฒน์"), (270, "ภูมิ"), (262, "พีรวิชญ์"), (145, "มณฑล")]

for student_id, name in students:
    btree.insert(student_id, name)

# แสดงโครงสร้าง B-Tree
btree.traverse()



Level 0 Keys: [192]
Level 1 Keys: [130, 145]
Level 1 Keys: [262, 270]


2. แก้ไข class B-Tree ให้มีการเก็บจำนวน Entry สูงสุด และต่ำสุด แทนการใช้ get_min_keys และ get_max_keys

In [8]:
class BTreeNode:
    def __init__(self, leaf=True, min_keys=None, max_keys=None):
        self.leaf = leaf  # เป็นใบหรือไม่
        self.keys = []  # เก็บค่าของโหนด (เรียงจากน้อยไปมาก)
        self.data = []  # ข้อมูลที่สัมพันธ์กับ key
        self.children = []  # ลูกของโหนด (ถ้าไม่ใช่ใบ)
        self.min_keys = min_keys  # จำนวน key ต่ำสุดในโหนด
        self.max_keys = max_keys  # จำนวน key สูงสุดในโหนด

class BTree:
    def __init__(self, order):
        self.root = None  # โหนดราก
        self.order = order  # ระดับของ B-Tree
        self.min_keys = (order - 1) // 2  # จำนวน key ต่ำสุด
        self.max_keys = order - 1  # จำนวน key สูงสุดในโหนด

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode(min_keys=self.min_keys, max_keys=self.max_keys)
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False, min_keys=self.min_keys, max_keys=self.max_keys)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf, min_keys=self.min_keys, max_keys=self.max_keys)
        mid = (order - 1) // 2  # คำนวณค่ากลางที่ถูกต้อง

        # ดึงค่า mid ขึ้นไป parent
        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        # ก๊อบปี้ค่าที่เหลือไปยังโหนดใหม่
        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        # ตัดค่าที่เคลื่อนย้ายออกจากโหนดเดิม
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def traverse(self, node=None, level=0):
        if node is None:
            node = self.root
        print("Level", level, "Keys:", node.keys)
        if not node.leaf:
            for child in node.children:
                self.traverse(child, level + 1)

# เพิ่มข้อมูลนักศึกษา
btree = BTree(3)
students = [(192, "สุภัทรา"), (130, "พีรพัฒน์"), (270, "ภูมิ"), (262, "พีรวิชญ์"), (145, "มณฑล")]

for student_id, name in students:
    btree.insert(student_id, name)

# แสดงโครงสร้าง B-Tree
btree.traverse()


Level 0 Keys: [192]
Level 1 Keys: [130, 145]
Level 1 Keys: [262, 270]


### ผลการทดลองที่ 3

1. เขียนโปรแกรมเพื่อทดสอบการค้นหาข้อมูลใน B-Tree ตามข้อมูลที่ได้เพิ่มในการทดลองก่อนหน้า

In [16]:
class BTreeNode:
    def __init__(self, leaf=True, min_keys=None, max_keys=None):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []
        self.min_keys = min_keys
        self.max_keys = max_keys

class BTree:
    def __init__(self, order):
        self.root = None
        self.order = order
        self.min_keys = (order - 1) // 2
        self.max_keys = order - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode(min_keys=self.min_keys, max_keys=self.max_keys)
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False, min_keys=self.min_keys, max_keys=self.max_keys)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf, min_keys=self.min_keys, max_keys=self.max_keys)
        mid = (order - 1) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def search(self, key):
        """ค้นหาข้อมูลจาก key ที่กำหนด"""
        def _search_node(node, key):
            i = 0
            # หาตำแหน่งที่เหมาะสม
            while i < len(node.keys) and key > node.keys[i]:
                i += 1

            # ถ้าเจอ key ที่ตรงกัน
            if i < len(node.keys) and key == node.keys[i]:
                return (node, i)

            # ถ้าเป็นใบและไม่เจอ key
            if node.leaf:
                return None

            # ค้นหาในลูกที่เหมาะสม
            return _search_node(node.children[i], key)

        if self.root is None:
            return None

        result = _search_node(self.root, key)
        if result:
            node, index = result
            return node.data[index]
        return None

    def traverse(self, node=None, level=0):
        if node is None:
            node = self.root
        print("Level", level, "Keys:", node.keys)
        if not node.leaf:
            for child in node.children:
                self.traverse(child, level + 1)


# สร้าง B-Tree และเพิ่มข้อมูลนักศึกษา
btree = BTree(3)
students = [(192, "สุภัทรา"), (130, "พีรพัฒน์"), (270, "ภูมิ"), (262, "พีรวิชญ์"), (145, "มณฑล")]

for student_id, name in students:
    btree.insert(student_id, name)

# ทดสอบการค้นหาข้อมูล
search_results = [
    btree.search(192),  # ค้นหาข้อมูลนักศึกษาที่มีรหัส 192
    btree.search(130),  # ค้นหาข้อมูลนักศึกษาที่มีรหัส 130
    btree.search(270),  # ค้นหาข้อมูลนักศึกษาที่มีรหัส 270
    btree.search(262),  # ค้นหาข้อมูลนักศึกษาที่มีรหัส 262
    btree.search(145),  # ค้นหาข้อมูลนักศึกษาที่มีรหัส 145
    btree.search(100)   # ค้นหาข้อมูลที่ไม่มีอยู่
]

# แสดงผลลัพธ์ของการค้นหาทั้งหมด
search_results

['สุภัทรา', 'พีรพัฒน์', 'ภูมิ', 'พีรวิชญ์', 'มณฑล', None]

### ผลการทดลองที่ 4

1. เขียนโปรแกรมเพื่อทดสอบการแสดงข้อมูลใน B-Tree 

In [10]:
class BTreeNode:
    def __init__(self, leaf=True, min_keys=None, max_keys=None):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []
        self.min_keys = min_keys
        self.max_keys = max_keys

class BTree:
    def __init__(self, order):
        self.root = None
        self.order = order
        self.min_keys = (order - 1) // 2
        self.max_keys = order - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode(min_keys=self.min_keys, max_keys=self.max_keys)
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False, min_keys=self.min_keys, max_keys=self.max_keys)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf, min_keys=self.min_keys, max_keys=self.max_keys)
        mid = (order - 1) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def display(self):
        """แสดงโครงสร้างของ B-Tree"""
        def _display(node, level):
            if node:
                # แสดง key และ data ในโหนดปัจจุบัน
                print('  ' * level + f"Keys: {node.keys}")
                print('  ' * level + f"Data: {node.data}")
                print('  ' * level + f"Is Leaf: {node.leaf}")
                print('  ' * level + f"Number of children: {len(node.children)}")
                print()
                # แสดงลูกทุกตัวแบบ recursive
                for child in node.children:
                    _display(child, level + 1)
        
        print("B-Tree Structure:")
        _display(self.root, 0)

# สร้าง B-Tree และเพิ่มข้อมูลนักศึกษา
btree = BTree(3)
students = [(192, "สุภัทรา"), (130, "พีรพัฒน์"), (270, "ภูมิ"), (262, "พีรวิชญ์"), (145, "มณฑล")]

for student_id, name in students:
    btree.insert(student_id, name)

# ทดสอบการแสดงผล B-Tree
btree.display()


B-Tree Structure:
Keys: [192]
Data: ['สุภัทรา']
Is Leaf: False
Number of children: 2

  Keys: [130, 145]
  Data: ['พีรพัฒน์', 'มณฑล']
  Is Leaf: True
  Number of children: 0

  Keys: [262, 270]
  Data: ['พีรวิชญ์', 'ภูมิ']
  Is Leaf: True
  Number of children: 0



### แบบฝึกหัด

1. ให้นักศึกษาเพิ่มเมธอดสำหรับลบข้อมูล (key และ data) ออกจาก B-Tree

In [2]:
class BTreeNode:
    def __init__(self, order, is_leaf=False):
        self.order = order
        self.is_leaf = is_leaf
        self.items = []  # List of (key, data)
        self.children = []  # List of child nodes

class BTree:
    def __init__(self, order):
        self.order = order
        self.root = BTreeNode(order, is_leaf=True)

    def insert(self, key, data):
        # โค้ดสำหรับการแทรกข้อมูล
        current_node = self.root
        while not current_node.is_leaf:
            current_node = current_node.children[0]  # Example traversal to find leaf
        current_node.items.append((key, data))
        current_node.items.sort(key=lambda x: x[0])  # Sort items by key

    def search(self, key):
        # โค้ดสำหรับการค้นหา
        current_node = self.root
        while current_node:
            for item in current_node.items:
                if item[0] == key:
                    return item[1]
            if current_node.is_leaf:
                break
            else:
                current_node = current_node.children[0]  # Example traversal to next level
        return None

    def delete(self, key):
        # ลบข้อมูลจาก B-Tree
        def delete_from_node(node, key):
            if not node:
                return None
            # ลบข้อมูลจาก node
            for i, item in enumerate(node.items):
                if item[0] == key:
                    del node.items[i]
                    return node
            return None
        
        delete_from_node(self.root, key)

    def display_all(self):
        # แสดงข้อมูลทั้งหมดใน B-Tree เรียงตาม key
        def inorder_traversal(node):
            if node:
                for child in node.children:
                    inorder_traversal(child)
                if node.is_leaf:
                    for item in node.items:
                        print(f"รหัสนักศึกษา: {item[0]} | ชื่อ: {item[1]['name']} | GPA: {item[1]['gpa']} | วิชาที่ลงทะเบียน: {', '.join(item[1]['courses'])}")
        inorder_traversal(self.root)

# สร้าง B-Tree สำหรับระบบทะเบียน
registration_system = BTree(order=3)

# เพิ่มข้อมูลนักศึกษา
def register_student(student_id, info):
    registration_system.insert(student_id, {
        "name": info["name"],
        "gpa": info["gpa"],
        "courses": info["courses"]
    })

# เพิ่มข้อมูลตัวอย่าง
register_student(6301, {
    "name": "สมชาย ใจดี",
    "gpa": 3.75,
    "courses": ["CS101", "CS102"]
})

register_student(6302, {
    "name": "สมหญิง รักเรียน",
    "gpa": 3.85,
    "courses": ["CS101", "MATH101"]
})

# ค้นหาข้อมูลนักศึกษา
def get_student_info(student_id):
    student = registration_system.search(student_id)
    if student:
        print(f"รหัสนักศึกษา: {student_id}")
        print(f"ชื่อ: {student['name']}")
        print(f"เกรดเฉลี่ย: {student['gpa']}")
        print(f"วิชาที่ลงทะเบียน: {', '.join(student['courses'])}")
    else:
        print(f"ไม่พบข้อมูลนักศึกษารหัส {student_id}")

# ทดสอบค้นหา
get_student_info(6301)

# ทดสอบการลบข้อมูล
registration_system.delete(6302)
get_student_info(6302)

# แสดงข้อมูลทั้งหมด
registration_system.display_all()


รหัสนักศึกษา: 6301
ชื่อ: สมชาย ใจดี
เกรดเฉลี่ย: 3.75
วิชาที่ลงทะเบียน: CS101, CS102
ไม่พบข้อมูลนักศึกษารหัส 6302
รหัสนักศึกษา: 6301 | ชื่อ: สมชาย ใจดี | GPA: 3.75 | วิชาที่ลงทะเบียน: CS101, CS102


2. ให้นักศึกษาเพิ่มเมธอดสำหรับอัปเดตข้อมูล (data) สำหรับ key ที่กำหนด

In [3]:
class BTreeNode:
    def __init__(self, order, is_leaf=False):
        self.order = order
        self.is_leaf = is_leaf
        self.items = []  # List of (key, data)
        self.children = []  # List of child nodes

class BTree:
    def __init__(self, order):
        self.order = order
        self.root = BTreeNode(order, is_leaf=True)

    def insert(self, key, data):
        # โค้ดสำหรับการแทรกข้อมูล
        current_node = self.root
        while not current_node.is_leaf:
            current_node = current_node.children[0]  # Example traversal to find leaf
        current_node.items.append((key, data))
        current_node.items.sort(key=lambda x: x[0])  # Sort items by key

    def search(self, key):
        # โค้ดสำหรับการค้นหา
        current_node = self.root
        while current_node:
            for item in current_node.items:
                if item[0] == key:
                    return item[1]
            if current_node.is_leaf:
                break
            else:
                current_node = current_node.children[0]  # Example traversal to next level
        return None

    def delete(self, key):
        # ลบข้อมูลจาก B-Tree
        def delete_from_node(node, key):
            if not node:
                return None
            # ลบข้อมูลจาก node
            for i, item in enumerate(node.items):
                if item[0] == key:
                    del node.items[i]
                    return node
            return None
        
        delete_from_node(self.root, key)

    def update(self, key, new_data):
        # อัปเดตข้อมูลของ key ที่กำหนด
        def update_in_node(node, key, new_data):
            for i, item in enumerate(node.items):
                if item[0] == key:
                    node.items[i] = (key, new_data)  # อัปเดตข้อมูลที่พบ
                    return True
            return False
        
        updated = update_in_node(self.root, key, new_data)
        if not updated:
            print(f"ไม่พบข้อมูลสำหรับรหัส {key}")

    def display_all(self):
        # แสดงข้อมูลทั้งหมดใน B-Tree เรียงตาม key
        def inorder_traversal(node):
            if node:
                for child in node.children:
                    inorder_traversal(child)
                if node.is_leaf:
                    for item in node.items:
                        print(f"รหัสนักศึกษา: {item[0]} | ชื่อ: {item[1]['name']} | GPA: {item[1]['gpa']} | วิชาที่ลงทะเบียน: {', '.join(item[1]['courses'])}")
        inorder_traversal(self.root)


# สร้าง B-Tree สำหรับระบบทะเบียน
registration_system = BTree(order=3)

# เพิ่มข้อมูลนักศึกษา
def register_student(student_id, info):
    registration_system.insert(student_id, {
        "name": info["name"],
        "gpa": info["gpa"],
        "courses": info["courses"]
    })

# เพิ่มข้อมูลตัวอย่าง
register_student(6301, {
    "name": "สมชาย ใจดี",
    "gpa": 3.75,
    "courses": ["CS101", "CS102"]
})

register_student(6302, {
    "name": "สมหญิง รักเรียน",
    "gpa": 3.85,
    "courses": ["CS101", "MATH101"]
})

# ค้นหาข้อมูลนักศึกษา
def get_student_info(student_id):
    student = registration_system.search(student_id)
    if student:
        print(f"รหัสนักศึกษา: {student_id}")
        print(f"ชื่อ: {student['name']}")
        print(f"เกรดเฉลี่ย: {student['gpa']}")
        print(f"วิชาที่ลงทะเบียน: {', '.join(student['courses'])}")
    else:
        print(f"ไม่พบข้อมูลนักศึกษารหัส {student_id}")

# ทดสอบค้นหา
get_student_info(6301)

# ทดสอบการอัปเดตข้อมูล
registration_system.update(6301, {
    "name": "สมชาย ใจดี",
    "gpa": 3.95,
    "courses": ["CS101", "CS102", "ENG101"]
})
get_student_info(6301)

# แสดงข้อมูลทั้งหมด
registration_system.display_all()


รหัสนักศึกษา: 6301
ชื่อ: สมชาย ใจดี
เกรดเฉลี่ย: 3.75
วิชาที่ลงทะเบียน: CS101, CS102
รหัสนักศึกษา: 6301
ชื่อ: สมชาย ใจดี
เกรดเฉลี่ย: 3.95
วิชาที่ลงทะเบียน: CS101, CS102, ENG101
รหัสนักศึกษา: 6301 | ชื่อ: สมชาย ใจดี | GPA: 3.95 | วิชาที่ลงทะเบียน: CS101, CS102, ENG101
รหัสนักศึกษา: 6302 | ชื่อ: สมหญิง รักเรียน | GPA: 3.85 | วิชาที่ลงทะเบียน: CS101, MATH101


3. ให้นักศึกษาเพิ่มเมธอดสำหรับแสดงข้อมูลทั้งหมดใน B-Tree เรียงตาม key

In [ ]:
class BTreeNode:
    def __init__(self, order, is_leaf=False):
        self.order = order
        self.is_leaf = is_leaf
        self.items = []  # List of (key, data)
        self.children = []  # List of child nodes

class BTree:
    def __init__(self, order):
        self.order = order
        self.root = BTreeNode(order, is_leaf=True)

    def insert(self, key, data):
        # โค้ดสำหรับการแทรกข้อมูล
        current_node = self.root
        while not current_node.is_leaf:
            current_node = current_node.children[0]  # Example traversal to find leaf
        current_node.items.append((key, data))
        current_node.items.sort(key=lambda x: x[0])  # Sort items by key

    def search(self, key):
        # โค้ดสำหรับการค้นหา
        current_node = self.root
        while current_node:
            for item in current_node.items:
                if item[0] == key:
                    return item[1]
            if current_node.is_leaf:
                break
            else:
                current_node = current_node.children[0]  # Example traversal to next level
        return None

    def delete(self, key):
        # ลบข้อมูลจาก B-Tree
        def delete_from_node(node, key):
            if not node:
                return None
            # ลบข้อมูลจาก node
            for i, item in enumerate(node.items):
                if item[0] == key:
                    del node.items[i]
                    return node
            return None
        
        delete_from_node(self.root, key)

    def update(self, key, new_data):
        # อัปเดตข้อมูลของ key ที่กำหนด
        def update_in_node(node, key, new_data):
            for i, item in enumerate(node.items):
                if item[0] == key:
                    node.items[i] = (key, new_data)  # อัปเดตข้อมูลที่พบ
                    return True
            return False
        
        updated = update_in_node(self.root, key, new_data)
        if not updated:
            print(f"ไม่พบข้อมูลสำหรับรหัส {key}")

    def display_all(self):
        # แสดงข้อมูลทั้งหมดใน B-Tree เรียงตาม key
        def inorder_traversal(node):
            if node:
                for child in node.children:
                    inorder_traversal(child)
                if node.is_leaf:
                    for item in node.items:
                        print(f"รหัสนักศึกษา: {item[0]} | ชื่อ: {item[1]['name']} | GPA: {item[1]['gpa']} | วิชาที่ลงทะเบียน: {', '.join(item[1]['courses'])}")
        inorder_traversal(self.root)


# สร้าง B-Tree สำหรับระบบทะเบียน
registration_system = BTree(order=3)

# เพิ่มข้อมูลนักศึกษา
def register_student(student_id, info):
    registration_system.insert(student_id, {
        "name": info["name"],
        "gpa": info["gpa"],
        "courses": info["courses"]
    })

# เพิ่มข้อมูลตัวอย่าง
register_student(6301, {
    "name": "สมชาย ใจดี",
    "gpa": 3.75,
    "courses": ["CS101", "CS102"]
})

register_student(6302, {
    "name": "สมหญิง รักเรียน",
    "gpa": 3.85,
    "courses": ["CS101", "MATH101"]
})

# ค้นหาข้อมูลนักศึกษา
def get_student_info(student_id):
    student = registration_system.search(student_id)
    if student:
        print(f"รหัสนักศึกษา: {student_id}")
        print(f"ชื่อ: {student['name']}")
        print(f"เกรดเฉลี่ย: {student['gpa']}")
        print(f"วิชาที่ลงทะเบียน: {', '.join(student['courses'])}")
    else:
        print(f"ไม่พบข้อมูลนักศึกษารหัส {student_id}")

# ทดสอบค้นหา
get_student_info(6301)

# แสดงข้อมูลทั้งหมด
print("\nข้อมูลทั้งหมดใน B-Tree เรียงตามรหัสนักศึกษา:")
registration_system.display_all()


รหัสนักศึกษา: 6301
ชื่อ: สมชาย ใจดี
เกรดเฉลี่ย: 3.75
วิชาที่ลงทะเบียน: CS101, CS102

ข้อมูลทั้งหมดใน B-Tree เรียงตามรหัสนักศึกษา:
รหัสนักศึกษา: 6301 | ชื่อ: สมชาย ใจดี | GPA: 3.75 | วิชาที่ลงทะเบียน: CS101, CS102
รหัสนักศึกษา: 6302 | ชื่อ: สมหญิง รักเรียน | GPA: 3.85 | วิชาที่ลงทะเบียน: CS101, MATH101


4. ให้นักศึกษาเพิ่มเมธอดสำหรับค้นหาข้อมูลแบบช่วง (range search)

In [5]:
class BTreeNode:
    def __init__(self, order, is_leaf=False):
        self.order = order
        self.is_leaf = is_leaf
        self.items = []  # List of (key, data)
        self.children = []  # List of child nodes

class BTree:
    def __init__(self, order):
        self.order = order
        self.root = BTreeNode(order, is_leaf=True)

    def insert(self, key, data):
        # โค้ดสำหรับการแทรกข้อมูล
        current_node = self.root
        while not current_node.is_leaf:
            current_node = current_node.children[0]  # Example traversal to find leaf
        current_node.items.append((key, data))
        current_node.items.sort(key=lambda x: x[0])  # Sort items by key

    def search(self, key):
        # โค้ดสำหรับการค้นหา
        current_node = self.root
        while current_node:
            for item in current_node.items:
                if item[0] == key:
                    return item[1]
            if current_node.is_leaf:
                break
            else:
                current_node = current_node.children[0]  # Example traversal to next level
        return None

    def delete(self, key):
        # ลบข้อมูลจาก B-Tree
        def delete_from_node(node, key):
            if not node:
                return None
            # ลบข้อมูลจาก node
            for i, item in enumerate(node.items):
                if item[0] == key:
                    del node.items[i]
                    return node
            return None
        
        delete_from_node(self.root, key)

    def update(self, key, new_data):
        # อัปเดตข้อมูลของ key ที่กำหนด
        def update_in_node(node, key, new_data):
            for i, item in enumerate(node.items):
                if item[0] == key:
                    node.items[i] = (key, new_data)  # อัปเดตข้อมูลที่พบ
                    return True
            return False
        
        updated = update_in_node(self.root, key, new_data)
        if not updated:
            print(f"ไม่พบข้อมูลสำหรับรหัส {key}")

    def display_all(self):
        # แสดงข้อมูลทั้งหมดใน B-Tree เรียงตาม key
        def inorder_traversal(node):
            if node:
                for child in node.children:
                    inorder_traversal(child)
                if node.is_leaf:
                    for item in node.items:
                        print(f"รหัสนักศึกษา: {item[0]} | ชื่อ: {item[1]['name']} | GPA: {item[1]['gpa']} | วิชาที่ลงทะเบียน: {', '.join(item[1]['courses'])}")
        inorder_traversal(self.root)

    def range_search(self, start_key, end_key):
        # ค้นหาข้อมูลภายในช่วงของ keys
        results = []

        # การเดินทางแบบ in-order และกรองข้อมูลที่อยู่ในช่วงที่กำหนด
        def inorder_traversal(node):
            if node:
                for child in node.children:
                    inorder_traversal(child)
                if node.is_leaf:
                    for item in node.items:
                        if start_key <= item[0] <= end_key:
                            results.append(item)
        inorder_traversal(self.root)
        
        if results:
            for item in results:
                print(f"รหัสนักศึกษา: {item[0]} | ชื่อ: {item[1]['name']} | GPA: {item[1]['gpa']} | วิชาที่ลงทะเบียน: {', '.join(item[1]['courses'])}")
        else:
            print(f"ไม่พบข้อมูลในช่วงระหว่าง {start_key} และ {end_key}")

# สร้าง B-Tree สำหรับระบบทะเบียน
registration_system = BTree(order=3)

# เพิ่มข้อมูลนักศึกษา
def register_student(student_id, info):
    registration_system.insert(student_id, {
        "name": info["name"],
        "gpa": info["gpa"],
        "courses": info["courses"]
    })

# เพิ่มข้อมูลตัวอย่าง
register_student(6301, {
    "name": "สมชาย ใจดี",
    "gpa": 3.75,
    "courses": ["CS101", "CS102"]
})

register_student(6302, {
    "name": "สมหญิง รักเรียน",
    "gpa": 3.85,
    "courses": ["CS101", "MATH101"]
})

register_student(6303, {
    "name": "สมพร ขยัน",
    "gpa": 3.95,
    "courses": ["CS102", "CS103"]
})

# ค้นหาข้อมูลนักศึกษา
def get_student_info(student_id):
    student = registration_system.search(student_id)
    if student:
        print(f"รหัสนักศึกษา: {student_id}")
        print(f"ชื่อ: {student['name']}")
        print(f"เกรดเฉลี่ย: {student['gpa']}")
        print(f"วิชาที่ลงทะเบียน: {', '.join(student['courses'])}")
    else:
        print(f"ไม่พบข้อมูลนักศึกษารหัส {student_id}")

# ทดสอบค้นหาช่วง
print("ข้อมูลนักศึกษาที่มีรหัสในช่วง 6301 ถึง 6302:")
registration_system.range_search(6301, 6302)


ข้อมูลนักศึกษาที่มีรหัสในช่วง 6301 ถึง 6302:
รหัสนักศึกษา: 6301 | ชื่อ: สมชาย ใจดี | GPA: 3.75 | วิชาที่ลงทะเบียน: CS101, CS102
รหัสนักศึกษา: 6302 | ชื่อ: สมหญิง รักเรียน | GPA: 3.85 | วิชาที่ลงทะเบียน: CS101, MATH101
